# My first kaggle competitions.
[House Sales in King County, USA] KuwaKuwa '02/28/2018

In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools 
from sklearn import linear_model 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

### Load the dataset.

Memo.  
https://www.kaggle.com/harlfoxem/housesalesprediction/data

In [2]:
#df_data = pd.read_csv("kc_house_data.csv")
df_data = pd.read_csv('../input/kc_house_data.csv')

In [3]:
#df_data["price"] = df_data["price"] / 10**6 
#df_data["sqft_living"] = df_data["sqft_living"] / 10**3 
#df_data["sqft_above"] = df_data["sqft_above"] / 10**3 
#df_data["sqft_living15"] = df_data["sqft_living15"] / 10**3 
#df_data["sqft_lot"] = df_data["sqft_lot"] / 10**3 
#df_data["sqft_lot15"] = df_data["sqft_lot15"] / 10**3 
#df_data["sqft_basement"] = df_data["sqft_basement"] / 10**3 

# df_data = df_data.drop(["id","date","zipcode"], axis=1) 

In [3]:
df_data.info()

Ref.  
[https://www.kaggle.com/harlfoxem/housesalesprediction/data] Column Metadata  
id				a notation for a house   
Data 			Date house was sold   
price			Price is prediction target <--(*)    
Bedrooms 		Number of Bedrooms/House   
bathrooms		Number of bathrooms/bedrooms   
sqft_living 	square footage of the home   
sqft_lot 		square footage of the lot   
floors			Total floors (levels) in house   
waterfront		House which has a view to a waterfront   
view 			Has been viewed   
condition		How good the condition is ( Overall )   
grade			overall grade given to the housing unit, based on King County grading system   
sqft_above		square footage of house apart from basement   
sqft_basement	square footage of the basement   
yr_built		Built Year   
yr_renovated	Year when house was renovated   
zipcode			zip   
lat				Latitude coordinate   
long			Longitude coordinate   
sqft_living15	Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area   
sqft_lot15		lotSize area in 2015(implies-- some renovations)   

In [5]:
display(df_data.shape)
display(df_data.head())
display(df_data.tail())

In [29]:
df_data.describe() 

Memo.  
・sqftはどんな単位？ ->[http://www.deaiteam.com/deaiteamblog/9845/] 1000sqftで約30坪。  
・lotとは？ ->敷地面積のよう。  
・gradeはどのような基準で決めている？ ->[King County grading system] に基づく。ある程度信頼できるのか。  
・sqft_aboveとは？ ->地下から離れた家の広さといった感じか。  
・latは緯度(latitude)。longは経度(longitude)。そもそもアメリカってどこの住宅費が高い？ ->[http://blogos.com/article/218263/] 高いのはLA,NY,WA,沿岸部といったところ。内陸部の住宅費は安そう。  
・sqft_living15,sqft_lot15の15って何を表している？sqft_living,sqft_lotとの違いは？ ->2015年時点のデータ。建築面積は敷地面積に影響を与えた可能性がある。  

### Predict explanatory variables.

In [30]:
df_data.corr()

In [18]:
k = 10
corrmat = df_data.corr()
corrmat['price'].sort_values(ascending = False)

Memo.  
相関関係が大きい変数。
1. sqft_living(0.702035)
2. grade(0.667434)
3. sqft_above(0.605567)
4. sqft_living15(0.585379)
5. bathrooms(0.525138)

In [8]:
# Stepwise
import pyper 
df_data_tmp = df_data.drop(["id","date","zipcode"], axis=1) 
r = pyper.R(use_pandas='True')
r.assign('df_data_tmp', df_data_tmp)
print(r("step(lm(price~.,data=df_data_tmp))"))

Memo.  
StepwiseによるAICが一番小さい組み合わせ。  
Step:  AIC=-68974.95  
price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors +   
waterfront + view + condition + grade + sqft_above + yr_built +   
    yr_renovated + lat + long + sqft_living15 + sqft_lot15  
変数が選びきれていない感じ。。

In [9]:
#全通りのAICを算出
import statsmodels.api as sm 

count = 1
for i in range(5):
    combi = itertools.combinations(df_data.drop(["price","id","date","zipcode"],axis=1).columns, i+1) 
    for v in combi:
        y = df_data["price"]
        X = sm.add_constant(df_data[list(v)])
        model = sm.OLS(y, X).fit()
        if count == 1:
            min_aic = model.aic
            min_var = list(v)
        if min_aic > model.aic:
            min_aic = model.aic
            min_var = list(v)
        count += 1
#        print("AIC:",round(model.aic), "variable:",list(v))
print("====minimam AIC====")
print(min_var,min_aic)

Memo.  
====minimam AIC====  
['sqft_living', 'waterfront', 'grade', 'yr_built', 'lat'] -6214.98880309  

In [ ]:
Memo.  
説明変数として、使えそうな変数を予測。

1)相関関係の大きい(0.6以上)、下記3変数。
sqft_living(0.702035)
grade(0.667434)
sqft_above(0.605567)

2)AICが最も小さい組み合わせの(6214.98880309)、下記5変数。
sqft_living
waterfront
grade
yr_built
lat

### View some graphs.

In [10]:
y_var = "price"
X_var = ["sqft_living","grade","sqft_above","waterfront","yr_built","lat"]
df = df_data[[y_var]+ X_var]

pd.plotting.scatter_matrix(df,alpha=0.3,s=10, figsize=(15,15))
plt.show()

### Confirm a strong relationship between 'price' and explanatory variables.

In [11]:
import itertools
key=['price']
df_data_trim = df_data[['sqft_living','grade','sqft_above','waterfront','yr_built','lat']]
li_combi = list(itertools.product(key,df_data_trim.columns))
for X,Y in li_combi:
    print("X=%s"%X,"Y=%s"%Y)
    df_data.plot(kind="scatter",x=X,y=Y,alpha=0.7,s=10,c="price",colormap="winter")
    plt.xlabel(X)
    plt.ylabel(Y)
    plt.tight_layout()
    plt.show()

### Confirm missing values.

In [9]:
pd.DataFrame(df_data.isnull().sum(), columns=["num of missing"])

Memo.  
There are nothing of missing values.

### Confirm abnormal values.

In [12]:
# sqft_living
df_data.plot(kind="scatter",x="price",y="sqft_living")
plt.show()

In [11]:
# grade
df_data.boxplot(column="price",by="grade")
plt.title("")
plt.ylabel("price")
plt.show()

In [13]:
# sqft_above
df_data.plot(kind="scatter",x="price",y="sqft_above")
plt.show()

In [14]:
# waterfront
df_data.boxplot(column="price",by="waterfront")
plt.title("")
plt.ylabel("price")
plt.show()

In [ ]:
# yr_built
df_data.plot(kind="scatter",x="price",y="yr_built")
plt.show()

In [ ]:
# lat
df_data.plot(kind="scatter",x="price",y="lat")
plt.show()

Memo.  
'price'が6$MM以上のデータが怪しいが、正則化したら問題ない？

### Predict again explanatory variables.

In [5]:
# 1）Multicollinearity(多重共線性).
y=df_data["price"].values
X=df_data[["sqft_living","grade","sqft_above"]].values
regr = LinearRegression(fit_intercept=True)
regr.fit(X, y)
print("coefficient of determination=%s"%regr.score(X,y))
print("slope=%s"%regr.coef_,"intercept=%s"%regr.intercept_)

In [8]:
# 1）Multicollinearity(多重共線性).
y=df_data["price"].values
X=df_data[["sqft_living","grade"]].values
regr = LinearRegression(fit_intercept=True)
regr.fit(X, y)
print("coefficient of determination=%s"%regr.score(X,y))
print("slope=%s"%regr.coef_,"intercept=%s"%regr.intercept_)

In [6]:
# 2）Multicollinearity(多重共線性).
y=df_data["price"].values
X=df_data[["sqft_living","waterfront","grade","yr_built","lat"]].values
regr = LinearRegression(fit_intercept=True)
regr.fit(X, y)
print("coefficient of determination=%s"%regr.score(X,y))
print("slope=%s"%regr.coef_,"intercept=%s"%regr.intercept_)

In [9]:
# 2）Multicollinearity(多重共線性).
y=df_data["price"].values
X=df_data[["sqft_living","waterfront","grade","lat"]].values
regr = LinearRegression(fit_intercept=True)
regr.fit(X, y)
print("coefficient of determination=%s"%regr.score(X,y))
print("slope=%s"%regr.coef_,"intercept=%s"%regr.intercept_)

In [7]:
# VIF.
df = df_data.drop(["price","id","date","zipcode"],axis=1)
for cname in df.columns:  
    y=df[cname]
    X=df.drop(cname, axis=1)
    regr = LinearRegression(fit_intercept=True)
    regr.fit(X, y)
    rsquared = regr.score(X,y)
    print(cname,":" ,1/(1-np.power(rsquared,2)))

### Calculate evaluation index. 

In [6]:
# 重回帰分析
y = df_data["price"].values
X = df_data[['sqft_living','waterfront','grade','lat']].values

stdsc = StandardScaler()

kf = KFold(n_splits=5, random_state=1234, shuffle=True)
kf.get_n_splits(X, y)

df_result = pd.DataFrame()

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_std = stdsc.fit_transform(X_train)
    X_test_std = stdsc.transform(X_test)

    #---
    regr = LinearRegression(fit_intercept=True)
    regr.fit(X_train_std, y_train)
    y_pred = regr.predict(X_test_std)
    #---
    
    df = pd.DataFrame({"y_test":y_test, "y_pred":y_pred})
    df_result = pd.concat([df_result, df], axis=0)

y_test = df_result["y_test"]
y_pred = df_result["y_pred"]
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE=%s"%round(mse,3) )
print("RMSE=%s"%round(np.sqrt(mse), 3) )
print("MAE=%s"%round(mae,3) )

Memo.  
重回帰分析  
MSE=49141368842.467  
RMSE=221678.526  
MAE=141037.207   

In [7]:
# k近傍法
y = df_data["price"].values
X = df_data[['sqft_living','waterfront','grade','lat']].values

stdsc = StandardScaler()

kf = KFold(n_splits=5, random_state=1234, shuffle=True)
kf.get_n_splits(X, y)

df_result = pd.DataFrame()

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_std = stdsc.fit_transform(X_train)
    X_test_std = stdsc.transform(X_test)

    #---
    k = 3
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_std, y_train)
    y_pred = knn.predict(X_test_std)
    #---
        
    df = pd.DataFrame({"y_test":y_test, "y_pred":y_pred})
    df_result = pd.concat([df_result, df], axis=0)

y_test = df_result["y_test"]
y_pred = df_result["y_pred"]
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE=%s"%round(mse,3) )
print("RMSE=%s"%round(np.sqrt(mse), 3) )
print("MAE=%s"%round(mae,3) )

memo.  
k近傍法  
MSE=53054283447.546  
RMSE=230335.155  
MAE=128355.329  

In [ ]:
# SVM
from sklearn.svm import SVC

y = df_data["price"].values
X = df_data[['sqft_living','waterfront','grade','lat']].values

stdsc = StandardScaler()

kf = KFold(n_splits=5, random_state=1234, shuffle=True)
kf.get_n_splits(X, y)

df_result = pd.DataFrame()

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_std = stdsc.fit_transform(X_train)
    X_test_std = stdsc.transform(X_test)
    
    #---
    C = 5
    clf = SVC(C=C, kernel="linear")
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_test_std)
    #---
    
    df = pd.DataFrame({"y_test":y_test, "y_pred":y_pred})
    df_result = pd.concat([df_result, df], axis=0)

y_test = df_result["y_test"]
y_pred = df_result["y_pred"]
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE=%s"%round(mse,3) )
print("RMSE=%s"%round(np.sqrt(mse), 3) )
print("MAE=%s"%round(mae,3) )

Memo.    
SVM  
結果が返ってこず。。

In [9]:
# 決定木
from sklearn.tree import DecisionTreeClassifier, export_graphviz

y = df_data["price"].values
X = df_data[['sqft_living','waterfront','grade','lat']].values

stdsc = StandardScaler()

kf = KFold(n_splits=5, random_state=1234, shuffle=True)
kf.get_n_splits(X, y)

df_result = pd.DataFrame()

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_std = stdsc.fit_transform(X_train)
    X_test_std = stdsc.transform(X_test)
    
    #---
    clf = DecisionTreeClassifier(criterion="gini", max_depth=None, min_samples_split=3, min_samples_leaf=3, random_state=1234)
    clf = clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_test_std)
    #---
    
    df = pd.DataFrame({"y_test":y_test, "y_pred":y_pred})
    df_result = pd.concat([df_result, df], axis=0)

y_test = df_result["y_test"]
y_pred = df_result["y_pred"]
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE=%s"%round(mse,3) )
print("RMSE=%s"%round(np.sqrt(mse), 3) )
print("MAE=%s"%round(mae,3) )

Memo.  
決定木  
MSE=64776364800.026  
RMSE=254512.013  
MAE=136935.943  

In [5]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier

y = df_data["price"].values
X = df_data[['sqft_living','waterfront','grade','lat']].values

stdsc = StandardScaler()

kf = KFold(n_splits=5, random_state=1234, shuffle=True)
kf.get_n_splits(X, y)

df_result = pd.DataFrame()

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_std = stdsc.fit_transform(X_train)
    X_test_std = stdsc.transform(X_test)
    
    #---
    clf = RandomForestClassifier(n_estimators=10, max_depth=2, criterion="gini",
                                                 min_samples_leaf=2, min_samples_split=2, random_state=1234)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test_std)
    #---
    
    df = pd.DataFrame({"y_test":y_test, "y_pred":y_pred})
    df_result = pd.concat([df_result, df], axis=0)

y_test = df_result["y_test"]
y_pred = df_result["y_pred"]
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE=%s"%round(mse,3) )
print("RMSE=%s"%round(np.sqrt(mse), 3) )
print("MAE=%s"%round(mae,3) )

Memo.  
ランダムフォレスト  
MSE=198184235593.489  
RMSE=445178.88  
MAE=274937.032  